#### librerie e funzioni 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
import time
from time import process_time
from scipy.optimize import *
from scipy.optimize import minimize_scalar

# !pip install pyMCFSimplex 
from pyMCFSimplex import *

C:\Users\39327\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def genera_Q(alpha, u, q, dimensione, p):
    
    Q_diag = []

    for i in range(dimensione):
        Q_i = abs(random.uniform((-q[i] / u[i] * alpha), (q[i] / u[i] * alpha))) # generazione randomica 
        Q_diag.append(Q_i)

    Q = np.zeros((dimensione, dimensione))
    np.fill_diagonal(Q, Q_diag) # creazione matrice 

    num_entrate_zero = int(p * dimensione) 
    indici_zeri = np.random.choice(dimensione, num_entrate_zero, replace=False) # trova p*dimensione indici dei costi quadratici da mettere a zero 

    for idx in indici_zeri:
        Q[idx, idx] = 0
        # Q_diag[idx] = 0

    return Q #, Q_diag


def leggi_file_dimacs(nome_file):
    numero_nodi = 0
    numero_archi = 0
    u = []
    b = []
    q = []
    from_= []
    to_= []
    edges = []

    with open(nome_file, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) > 0:
                if parts[0] == 'p':  # parts[0] è il primo carattere di ogni riga - nel formato standard DIMACS può essere c, p, n, a
                    # legge il numero di nodi e archi dal problema
                    numero_nodi = int(parts[2])
                    numero_archi = int(parts[3])
                    # inizializza il vettore di supply con zeri
                    b = [0] * numero_nodi
                elif parts[0] == 'n':
                    # legge i valori di supply per i nodi
                    nodo_id = int(parts[1])
                    supply = int(parts[2])
                    # assegna il valore di supply al nodo corrispondente
                    b[nodo_id - 1] = supply
                elif parts[0] == 'a':
                    # leggi l'arco e il suo cotso
                    from_node = int(parts[1])
                    to_node = int(parts[2])
                    max_capacity = int(parts[4])
                    costo = int(parts[5])  # leggiamo costo corretto
                    from_.append(from_node)
                    to_.append(to_node)
                    u.append(max_capacity)
                    q.append(costo)
                    edges.append((from_node , to_node ))

    return numero_nodi, numero_archi, np.array(u), b, np.array(q), edges,from_, to_

# Questa funzione crea un nuovo file .dmx in cui i valori delle righe che iniziano con 'a' (definizione archi)
# vengono sostituiti con i valori presenti nel vettore gradient. L'obiettivo è modificare il file di input 
# (input_file) in modo da utilizzare al posto dei costi lineari del file originale, il gradiente 
# (vogliamo risolvere il sottoproblema lineare, aka l'approssimazione lineare del problema Taylor 1st ordine).   

def modify_file_with_gradient(input_file, output_file, gradient):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # inizalizzazione dell'indice per il vettore gradient
    gradient_index = 0

    # modifica delle righe che iniziano con 'a'
    for i in range(len(lines)):
        if lines[i].startswith('a'):
            words = lines[i].split()
            words[-1] = str(gradient[gradient_index])
            gradient_index += 1
            lines[i] = ' '.join(words)

    # creazione nuovo file
    with open(output_file, 'w') as file:
        file.writelines(lines)
        
# questo codice serve per calcolare lo step size trovando alpha che minimizzi f (xk + α(dk − xk))
# rispettando il vincolo 0 ≤ α ≤ 1. dove f è la nostra funzione di costo. 

# definizione della funzione quadratica - la nostra f iniziale x^TQx + q
def quadratic_function(alpha, xk, dk, Q, q):

    x_alpha = [x + alpha * (d - x) for x, d in zip(xk, dk)] 
    x_alpha = np.array(x_alpha) 

    # calcolo il valore della funzione obiettivo
    return x_alpha.T @ Q @ x_alpha + q @ x_alpha


def find_optimal_alpha(xk, dk, Q, q): # exact line search
   
    objective_function = lambda alpha: quadratic_function(alpha, xk, dk, Q, q) # nostra funzione obiettivo che dipende da alpha 
    result = minimize_scalar(objective_function, bounds=(0, 1), method = 'bounded') # soluzione del problema di minimo con vincolo su alpha che deve stare tra 0 e 1 

    return result.x
# MCF solver intern - do not change
def showModuleFunctionality(mcf):
    vettore_soluzione = {}  
    nmx = mcf.MCFnmax()
    mmx = mcf.MCFmmax()
    pn = mcf.MCFnmax()
    pm = mcf.MCFmmax()

    pU = []
    caps = new_darray(mmx)
    mcf.MCFUCaps(caps)
    for i in range(0, mmx):
        pU.append(darray_get(caps, i))

    pC = []
    costs = new_darray(mmx)
    mcf.MCFCosts(costs)
    for i in range(0, mmx):
        pC.append(darray_get(costs, i))

    pDfct = []
    supply = new_darray(nmx)
    mcf.MCFDfcts(supply)
    for i in range(0, nmx):
        pDfct.append(darray_get(supply, i))

    pSn = []
    pEn = []
    startNodes = new_uiarray(mmx)
    endNodes = new_uiarray(mmx)
    mcf.MCFArcs(startNodes, endNodes)
    for i in range(0, mmx):
        pSn.append(uiarray_get(startNodes, i) + 1)
        pEn.append(uiarray_get(endNodes, i) + 1)

    #print("arc flow")
    length = mcf.MCFm()
    flow = new_darray(length)
    length = mcf.MCFn()
    nms = new_uiarray(length)
    mcf.MCFGetX(flow, nms)

   

    for i in range(0, length):
        if uiarray_get(nms, i)== 4294967295:
            break
        else:
       # print("flow", darray_get(flow, i), "arc", uiarray_get(nms, i))
            vettore_soluzione[uiarray_get(nms, i)] = darray_get(flow, i)

    return vettore_soluzione  # restituisce il vettore_soluzione alla fine della funzione
def visualize(k, f_val, alpha, prodotto_scalare, k_granularity, tempo, end, found_optimal): 

    # la funzione stampa un summary delle seguenti quantità ad ogni iterazione (oppure ogni 'k_granularity' iterazioni) :
    # - iterazione corrente
    # - step size, 
    # - optimal solution, 
    # - prodotto scalare 

    # alla fine dell'esecuzione verrà mostrato :
    # - prodotto scalare finale,
    # - step size finale
    # - numero totale di iterazioni
    # - tempo di esecuzione totale 
    
    if end:
        print("_"*110)
        print("STOP: ")
        if found_optimal:
            print('Found optimal solution!')
        print('Final scalar product: {:>10.6f}  Final step size: {:>10.8f}  Total number iterations: {:>4}'.format(prodotto_scalare, alpha, k))
        print('Total running time: {:>10.2f} seconds'.format(tempo))    
        
    if  k % k_granularity == 0:
        print('Iteration: {:>4}   Step size: {:>10.8f}   Valore di f in x_k: {:>10.6f}   Scalar product grad, d: {:>10.2f} '.format(k, alpha, f_val, prodotto_scalare))
    return
class ProblemUnfeasibleError(Exception):
    pass
# away vertex
def away_step(grad, S):
    costs = {}
    
    for k,v in S.items():
        cost = np.dot(k,grad)
        costs[cost] = [k,v]
    vertex, alpha = costs[max(costs.keys())]  
    return vertex, alpha

# aggiornare active set come pseudocodice 
def update_S(S,gamma, Away, vertex):
    
    S = S.copy()
    vertex = tuple(vertex)
    
    if not Away:
        if vertex not in S.keys():
            S[vertex] = gamma
        else:
            S[vertex] *= (1-gamma)
            S[vertex] += gamma
            
        for k in S.keys():
            if k != vertex:
                S[k] *= (1-gamma)
    else:
        for k in S.keys():
            if k != vertex:
                S[k] *= (1+gamma)
            else:
                S[k] *= (1+gamma)
                S[k] -= gamma
    return {k:v for k,v in S.items() if np.round(v,3) > 0}
def visualize_bis(k, f_val, alpha, primal_gap_k, k_granularity): 

    # la funzione stampa un summary delle seguenti quantità ad ogni iterazione (oppure ogni 'k_granularity' iterazioni) :
    # - iterazione corrente
    # - step size, 
    # - optimal solution, 
    # - prodotto scalare 

    # alla fine dell'esecuzione verrà mostrato :
    # - prodotto scalare finale,
    # - step size finale
    # - numero totale di iterazioni
    # - tempo di esecuzione totale 
    
 
  
    
     if  k % k_granularity == 0:
        print('Iteration: {:>4}   Step size: {:>10.8f}   Valore di f in x_k: {:>10.6f}   Dual gap: {:>10.2f} '.format(k, alpha, f_val, primal_gap_k))
        return
def line_search_afw(x, d, gamma_max,func):
    
    def fun(gamma):
        ls = x + gamma*d
        return func(ls)

    res = minimize_scalar(fun, bounds=(0, gamma_max), method='bounded')

    gamma = res.x
    ls = x + gamma*d        
    return ls, gamma


#### FRANK WOLFE TRADIZIONALE 

In [5]:
import numpy as np
from datetime import datetime
import time

def FW_trad_modificato(b,n,from_,to,f_tol, time_tol, nome_file, epsilon, Q, q, u, numero_archi, x, step_size_ottimo=True, visualize_res=True):
    k = 0
    bestlb = -np.inf
    end = False
    now = datetime.now()
    primal_gap = []
    function_value = [func(x)]
    elapsed_time = [0]  # Cambiato il nome in "elapsed_time" per evitare conflitti
    tempo_per_it_MCF = [0]  # Nuova lista per registrare il tempo di esecuzione del solver MCF
    f_improv = np.inf
    tempi_per_it = [0]  # Registra il tempo totale per iterazione
    tempi_per_it_MCF = [0]  # Registra il tempo solo per l'iterazione del solver MCF
    #seed = 42
    #np.random.seed(seed)
    #x_old = []

    #for u_i in u:
     #   x_i = random.randint(0, u_i)
      #  x_old.append(x_i)

    while abs(f_improv) > f_tol and elapsed_time[-1] < time_tol:
        start = time.perf_counter()

        gradient = (2 * np.dot(Q, x)) + q
        start_MCF = time.perf_counter()
       # modify_file_with_gradient(nome_file, 'output.dmx', gradient)
        mcf = MCFSimplex()
        nmx     = n
        mmx     = numero_archi
        pn      = n
        pm      = numero_archi
        pU      = u.tolist()
        pC      = gradient.tolist()
        pDfct   = b
        pSn     =  to# column to
        pEn     = from_
     
        mcf.LoadNet(nmx, mmx, pn, pm, CreateDoubleArrayFromList(pU), CreateDoubleArrayFromList(pC),
            CreateDoubleArrayFromList(pDfct), CreateUIntArrayFromList(pSn),
            CreateUIntArrayFromList(pEn))
        
       # mcf.LoadDMX(inputStr)
        mcf.SolveMCF()
       # sol_x=risolvi_problema_ottimizzazione(n, m, u, b, gradient, edges, from_, to)
      

        end_MCF = time.perf_counter()  # Registro il tempo di fine per il solver MCF
        tempo_per_it_MCF.append(end_MCF - start_MCF)  # Calcolo il tempo per l'iterazione del solver MCF

        if mcf.MCFGetStatus() == 0:
            soluzione_ottima = mcf.MCFGetFO()
        else:
            raise ProblemUnfeasibleError("The problem is unfeasible.")

        vettore_soluzione = showModuleFunctionality(mcf)
        sol_x = [0] * numero_archi

        for key in vettore_soluzione:
           # if key <= 1000:
            sol_x[key] = vettore_soluzione[key]

        v = np.array(sol_x)
        u = np.array(u)
        d = v - x
     #   print('questo è d ',sum(d))

        if visualize_res and k > 0:
            #visualize_bis(k, function_value[-1], alpha, primal_gap[-1], k_granularity=1)
            print(f'iter: {k} iter, fval = {function_value[-1]}, fbest = {func(x):.8e}, gap = {primal_gap[-1]:.4e}')
        lb = func(x)+ np.dot(np.array(gradient), np.array(v) - np.array(x))  # Compute the lower bound
       
        if lb > bestlb:
            bestlb = lb
        
        gap = (func(x) - bestlb)/ max(abs(func(x)), 1)  # Compute the relative gap
        
      #  print(f'{i}\t{v:.8e}\t{bestlb:.8e}\t{gap:.4e}')
        
        if gap <= epsilon:  # Stopping criterion
            status = 'optimal'
            break
       # if np.abs(np.dot(-gradient, d)) <= epsilon:
        #    print('siiiiii')
        #    break
        else:
            primal_gap.append(gap)

        if step_size_ottimo:
            x, alpha = line_search_afw(x, d, 1, func)
        else:
            alpha = 2 / (2 + k)
            x = x + alpha * d

        end = time.perf_counter()
        tempo_it = end - start
        tempi_per_it.append(tempo_it)
        elapsed_time.append(elapsed_time[k] + tempo_it)  # Registriamo il tempo dell'iterazione
        f_improv = function_value[-1] - func(x)
        function_value.append(func(x))
        
        k += 1

        if abs(f_improv) < f_tol:
            print('Terminerei per f_tol', ' f improv: ' ,f_improv)
        if elapsed_time[-1] > time_tol:
            print('Terminerò per time')

    print('Final gap primale: {:>10.6f}  Final step size: {:>10.8f}  Total number iterations: {:>4}'.format(primal_gap[-1], alpha, k))
    print('Total running time: ', sum(tempi_per_it))
    
    return x, function_value, elapsed_time, primal_gap, k, tempi_per_it, tempo_per_it_MCF


### FRANK WOLFE AWAY STEP 

In [6]:
#AFW Algorithm
  
def AFW(nome_file, epsilon, max_iter, Q, q, numero_archi,func, k_granularity, x_old, visualize_res=True):
    function_value=[]
    k = 0    
    bestlb = -np.inf
    gamma = 1
    start_time = time.time()
    tempo_tot = 0
    prodotto_scalare = -float("inf")
    end = False
    found_optimal = False
    f_values = []
    tempi_per_it = [0]
    gap = []
    now = datetime.now()
    primal_gap = []
    function_value = [func(x_0)]
    elapsed_time = [0]  # Cambiato il nome in "elapsed_time" per evitare conflitti
    tempo_per_it_MCF = [0]  # Nuova lista per registrare il tempo di esecuzione del solver MCF
    f_improv = np.inf
    tempi_per_it = [0]  # Registra il tempo totale per iterazione
    tempi_per_it_MCF = [0]  # Registra il tempo solo per l'iterazione del solver MCF
 
    
    
    x_old=np.array(x_old)
  
    S = {tuple(x_old): 1} # active set inizializzazione
   

    while abs(f_improv) > f_tol and elapsed_time[-1] < time_tol:
   # while abs(prodotto_scalare)>= epsilon and k < max_iter: # stopping criteria

        start = time.perf_counter()

        f_x_current = np.dot(np.dot(x_old, Q), x_old) + np.dot(q, x_old)
        f_values.append(f_x_current)

        # STEP 1 : calcolo gradiente
        gradient = (2 * np.dot(Q, x_old)) + q
        # modify_file_with_gradient(nome_file, 'output.dmx', gradient)
        mcf = MCFSimplex()
        start_MCF = time.perf_counter()  # Registro il tempo di inizio per il solver MCF
        nmx     = n
        mmx     = numero_archi
        pn      = n
        pm      = numero_archi
        pU      = u.tolist()
        pC      = gradient.tolist()
        pDfct   = b
        pSn     = to# column to
        pEn     = from_
     
        mcf.LoadNet(nmx, mmx, pn, pm, CreateDoubleArrayFromList(pU), CreateDoubleArrayFromList(pC),
            CreateDoubleArrayFromList(pDfct), CreateUIntArrayFromList(pSn),
            CreateUIntArrayFromList(pEn))
        
       # mcf.LoadDMX(inputStr)
        mcf.SolveMCF()
       # sol_x=risolvi_problema_ottimizzazione(n, m, u, b, gradient, edges, from_, to)
      

        end_MCF = time.perf_counter()  # Registro il tempo di fine per il solver MCF
        tempo_per_it_MCF.append(end_MCF - start_MCF)  # Calcolo il tempo per l'iterazione del solver MCF

        if mcf.MCFGetStatus() == 0:
            soluzione_ottima = mcf.MCFGetFO()
        else:
            raise ProblemUnfeasibleError("The problem is unfeasible.")

        vettore_soluzione = showModuleFunctionality(mcf)
        sol_x = [0] * numero_archi

        for key in vettore_soluzione:
           # if key <= 1000:
            sol_x[key] = vettore_soluzione[key]

        # calcolo delle x_bar e determinazione della direzione di ricerca
        v = sol_x.copy()
    
        d_FW = v - x_old
        
        # calcolo away vertex e direzione d_A 
        a, alpha_a = away_step(gradient, S)
        d_A = x_old - a
       
        # Check se FW gap è maggiore dell'away gap --> per capire quale passo usare 
        if np.dot(-gradient, d_FW) >= np.dot(-gradient, d_A):
            # scegliamo FW direction
            d = d_FW
            vertex = v
            gamma_max = 1
            Away = False
        else:
            # scegliamo Away direction
            d = d_A
            vertex = a
            # ATTENZIONE ZERO DIVISION ERROR QUI 
            gamma_max = alpha_a/(1-alpha_a)
            # print('alpha AFW:', alpha_a)
        if visualize_res and k > 0:
            #visualize_bis(k, function_value[-1], alpha, primal_gap[-1], k_granularity=1)
            print(f'iter: {k} iter, fval = {function_value[-1]}, fbest = {func(x_old):.8e}, gap = {primal_gap[-1]:.4e}')
        lb = func(x_old)+ np.dot(np.array(gradient), np.array(d))  # Compute the lower bound
       
        if lb > bestlb:
            bestlb = lb
        
        gap = (func(x_old) - bestlb)/ max(abs(func(x_old)), 1)  # Compute the relative gap
        
      #  print(f'{i}\t{v:.8e}\t{bestlb:.8e}\t{gap:.4e}')
        
        if gap <= epsilon:  # Stopping criterion
            status = 'optimal'
            break
       # if np.abs(np.dot(-gradient, d)) <= epsilon:
        #    print('siiiiii')
        #    break
        else:
            primal_gap.append(gap)
        # ricerca di x_new e gamma usando line search 
        prodotto_scalare = np.dot(gradient, d)
        x_old, gamma = line_search_afw(x_old,d, gamma_max, func)

        # aggiornamento active set
        S = update_S(S,gamma, Away, vertex)


        #gradient_per_check = (2 * np.dot(Q, x_new) ) + q
   
        end = time.perf_counter()
        tempo_it = end - start
        tempi_per_it.append(tempo_it)
        elapsed_time.append(elapsed_time[k] + tempo_it)  # Registriamo il tempo dell'iterazione
        f_improv = function_value[-1] - func(x_old)
        function_value.append(func(x_old))
        
      
        k+=1

        
        if abs(f_improv) < f_tol:
            print('Terminerei per f_tol', ' f improv: ' ,f_improv)
        if elapsed_time[-1] > time_tol:
            print('Terminerò per time')
    print('Final gap primale: {:>10.6f}  Final step size: {:>10.8f}  Total number iterations: {:>4}'.format(primal_gap[-1], gamma, k))
    
    
    
    print('Total running time: ', sum(tempi_per_it))
    
    return function_value, k, tempo_tot, tempi_per_it, x_old, found_optimal,primal_gap,tempi_per_it, tempo_per_it_MCF,elapsed_time


### GUROBI 

In [ ]:
import gurobipy as gp
func = lambda x: np.dot(x, np.dot(Q, x))+np.dot(q,x)
nome_file = '1000/netgen-1000-1-1-a-a-ns.dmx'
n, m, u, b, q, _, from_ , to= leggi_file_dimacs(nome_file)
Q = genera_Q(100, u, q, numero_archi, 0.6)
E = np.zeros((n, m))

# Popola la matrice di incidenza E in base alle liste dei nodi entranti e uscenti
for j in range(m):
    i_entrante = to[j]
    i_uscente = from_[j]
    E[i_entrante - 1, j] = -1  # L'arco j è entrante nel nodo i_entrante
    E[i_uscente - 1, j] = 1    # L'arco j è uscente dal nodo i_uscente

model = gp.Model()

#ariabili di decisione + vincolo sulla x 
x = []
for i in range(m):
    x.append(model.addVar(lb=0.0, ub=u[i], name=f"x_{i}"))


# funzione obiettivo
model.setObjective(x @ Q @ x + q @ x, sense=gp.GRB.MINIMIZE)

# Vincoli 
# Vincoli di conservazione del flusso (Ex = b)

for i in range(n):
   
    model.addConstr(sum(E[i][j] * x[j] for j in range(m)) == b[i], name=f"flow_balance_{i}")
    

# Risoluzione del problema
model.optimize()

# Stampare il risultato
if model.status == gp.GRB.OPTIMAL:
    print("Soluzione ottima trovata:")
    for i in range(m):
        print(f"x[{i}] = {x[i].x}")
    print(f"Valore della funzione obiettivo: {model.objVal}")
else:
    print("Il problema non ha una soluzione ottimale.")

# Cleanup
model.dispose()


### ESPERIMENTI 

In [10]:
func = lambda x: np.dot(x, np.dot(Q, x))+np.dot(q,x)
nome_file_dmx = '1000/netgen-1000-1-1-a-a-ns.dmx'
n, numero_archi, u, b, q, _, from_ , to= leggi_file_dimacs(nome_file_dmx)
Q = genera_Q(10, u, q, numero_archi, 0.2)
x_0 = u/2   
f_tol =1e-9
time_tol = np.inf
epsilon = 0.001

x, function_value, elapsed_time, primal_gap, k, tempi_per_it, tempo_per_it_MCF = FW_trad_modificato(b,n,from_,to,f_tol, time_tol, nome_file_dmx, epsilon, Q, q, u, numero_archi,x_0, step_size_ottimo=True, visualize_res=True)
function_value, k, tempo_tot, tempi_per_it, x_old, found_optimal,primal_gap,tempi_per_it, tempo_per_it_MCF,elapsed_time= AFW(nome_file_dmx, epsilon = 0.001, max_iter = 1000, Q = Q, q = q, numero_archi = numero_archi, func=func, k_granularity=1, x_old = x_0)

iter: 1 iter, fval = 60954.41524949436, fbest = 6.09544152e+04, gap = 1.6462e+00
iter: 2 iter, fval = 54669.45439942324, fbest = 5.46694544e+04, gap = 8.1392e-01
iter: 3 iter, fval = 52510.76407048955, fbest = 5.25107641e+04, gap = 5.5146e-01
iter: 4 iter, fval = 51538.680414753115, fbest = 5.15386804e+04, gap = 3.2755e-01
iter: 5 iter, fval = 49852.8520016774, fbest = 4.98528520e+04, gap = 2.6390e-01
iter: 6 iter, fval = 49238.96643781049, fbest = 4.92389664e+04, gap = 2.1860e-01
iter: 7 iter, fval = 48607.46266604449, fbest = 4.86074627e+04, gap = 1.4982e-01
iter: 8 iter, fval = 48349.6477129409, fbest = 4.83496477e+04, gap = 1.3877e-01
iter: 9 iter, fval = 47984.58899866193, fbest = 4.79845890e+04, gap = 1.3298e-01
iter: 10 iter, fval = 47770.19543705501, fbest = 4.77701954e+04, gap = 1.2403e-01
iter: 11 iter, fval = 47369.285908168036, fbest = 4.73692859e+04, gap = 1.0350e-01
iter: 12 iter, fval = 47080.66009388717, fbest = 4.70806601e+04, gap = 9.5912e-02
iter: 13 iter, fval = 469